# II: Theory and practice

## Outcomes

You will understand:

* What a Bayesian is.
* The idea of a data generating process;
* The relation of models and parameters;
* What uncertainty is, where it comes from, and why it is important;
* The evolution of probabilistic programming languages;
* What probability is and how it represents uncertainty;
* The distinction between prediction and inference and the relation to forward and inverse probability.
* A high-speed review of basic probability theory through code: 
    * axioms of probability
    * mass functions, distributions, random variables
    * likelihood and sampling
    * joint, marginal, conditional, Bayes' rule
    * entropy, divergence
* Major classes of inference algorithms

## Poll

## What are Bayesian methods?

A Bayesian is someone who:

* Is happy to live without truth;
* Reasons from belief to belief, guided by evidence;
* Thinks backwards by thinking forwards.

### Without truth

### Belief to belief

### Forwards, not back


## Models: Data generating processes
Let's get back to computational interaction. We need *models* to do computational interaction, and they need to be *executable*. That means code that simulates or emulates some interaction
element of interest. At the heart of Bayesian modelling we have the idea of a **data generating process**, a process which we believe is generating data we observe.

[Image]



### The advance of programming languages

Advances in programming languages change the way we express models, and implicitly how we think about modelling the world. A model can be written as a *function*.

#### Traditional: Python, C, Java, Rust, C#, ...
We express models as operations on primitives, like numbers.

```python
def f(x):
    return x ** 2 + x - 2 
```

#### Vectorised: NumPy, eigen, Julia, ...
We express models directly over numerical arrays ("tensors"). Code gets shorter, cleaner, and more efficient, takes advantage of hardware, etc.

```python
import numpy as np
def f(x):
    return np.sum(x**2 + x - 2, axis=1)
```

### Differentiable: autograd, JAX, PyTorch, TensorFlow, ...
Defining a function over numerical arrays *automatically* also defines a function returning the partial derivatives. Universal first-order optimisation (gradient descent) becomes available.

* *Now we can write **inverse programs**: we define the output, and solve for the input*

In [ ]:
import autograd.numpy as anp
import autograd


def f(x):
    return anp.sum((x**2 + x - 2) ** 2, axis=1)


df = autograd.elementwise_grad(f)

In [ ]:
x = anp.eye(4)
print(f(x))
print(df(x))

In [ ]:
# if only we had some way of repeating
# things on a computer
x = x - df(x) * 0.1
print(f(x))
x = x - df(x) * 0.1
print(f(x))
x = x - df(x) * 0.1
print(f(x))
x = x - df(x) * 0.1
print(f(x))
x = x - df(x) * 0.1
print(f(x))
x = x - df(x) * 0.1
print(f(x))
x = x - df(x) * 0.1
print(f(x))
x = x - df(x) * 0.1
print(f(x))


### Probabilistic (PPLs): Stan, PyMC, Turing.jl, BUGS/JAGS, ...
This adds distributions to programs. It automatically includes *random simulation*, *likelihood* and most importantly *inference*.

We don't need a PPL to do random simulation (a **stochastic model**):


In [ ]:
import numpy as np
import scipy.stats as ss


# forward: random simulation
def f(x):
    x = np.array(x)
    y = np.random.normal(0, 1, x.shape)
    return x**2 + x - 2 + y


# different on every run
# models variation in the world
f([1, 2, 3])

or to do the inverse; compute the likelihood of (stochastic) observations under a parameterised model:

In [ ]:
def llik_f(x, y):
    x = np.array(x)
    # return log-lik of observations y under model settings x
    return np.sum(ss.norm(x**2 + x - 2, 1).logpdf(y))


print(llik_f([1, 2, 3], f([1, 2, 3])))

but to do the **inference** part -- to work out the relative probability of possible inputs that might have generated an observation -- we'd be much better off using a real probabilistic programming language. This will implement efficient algorithm to allow us to write **uncertain inverse programs**.


## Parameters and models

## Uncertainty

>    All theorems are true.  
>    All models are wrong.  
>    And all data are inaccurate.  
>    What are we to do?  
>    We must be sure to remain uncertain. 

-- *[Leonard A. Smith, Proc. International School of Physics ``Enrico Fermi", (1997)](http://www2.maths.ox.ac.uk/~lenny/fermi96_main_abs.html)* 

#### What is uncertainty and where does it come from?


## What is a Bayesian [II]?

A Bayesian:

* Represents, preserves and manipulates uncertainty about unknown states. Uncertainty is **first-class**.
* Builds generative models of the phenomena under consideration, that simulate plausible observations.
* Reasons about the unknown parameters that modulate the behaviour of those generative models.


## Probability



### What is probability?

A fraught philosophical question! See the references for debates on this topic. We'll make some uncontroversial statements, then an *interpretation* of probability.

**Probability, as we shall use it, is simply an extension of ordinary logic to uncertain situations.**


#### Basic facts

* We associate numbers (probabilities) with sets (*events*), written $P(A)$to mean "probability of event A".
* Probabilities are non-negative and cannot exceed 1: $0 \leq P(A) \leq 1$ 
* A probability distribution associates a set of distinct *outcomes* to probabilities. $P(X=x)$ meaning the probability that variable $X$ takes on outcome $x$.
* An *event* is any set of outcomes.
* The probability of all possible outcomes in a distribution sums to 1 exactly.
* The probability of any set of disjoint events that cover all outcomes is therefore also 1.
* => If A and B are events $P(A \lor B) = P(A) + P(B) - P(A\land B)$ (sum rule; A and B are sets of outcomes)
* => If A has probability $P(A)=P(X \in A)$, $P(¬A)=P(X \notin A)=1-P(A)$
* The probability of two *independent* events A and B is $P(A \land B) = P(A)P(B)$
* The probability of A *given we know that* an event B is true is written $P(A|B) = P(A \land B)/P(B)$
* The probability of P(A|B) is **not** in general P(B|A).
* $P(A|B) = P(B|A)P(A) / P(B)$ (Bayes' Rule)
* $P(B) = \sum_B P(B|A)P(A)$ 

---

Scenario: modelling the age of someone's bike.

* The probability of a person's bike being more than 5 years old is an *event*; perhaps $P(age>5)=0.5$
* A bike's age cannot be less likely than impossible or more likely than certain.
* A probability distribution might map a finite range of integer ages to probabilities; real numbers in the range 0-1; perhaps $P(age=0)=0.02$,  $P(age=1)=0.2$, $P(age=3)=0.1$, etc.
    * Note: it doesn't have to be this way; we could have a continuous age, or a discrete but infinite set of ages. The details get a bit more fiddly.
* A bike being less than five years old $P(age<5)=P(age \in \{0,1,2,3,4\})$, or being an even number of years old $P(age \mod 2=0)=P(age \in \{0, 2, 4, 6, \dots \})$, or being 1 year old $P(age=0)=P(age \in \{0\})$; these are all *events*.
* The probability of a bike "having" an age is tautologically 1 in this model; so the probability of each outcome $P(age=1)+P(age=2)+P(age=3)+\dots=1.0$
* The probability of a bike being less than five years old *or* being equal to or more than five years old must be 1.0, by the same logic (every outcome is covered exactly once). $P(age<5) + P(age\geq 5)=1.0$
* The probability of a bike being less than five years old or more than three years old is: $P(age>3 | age<5) = P(age<5) + P(age>3) - P(age<5 \land age>3)$ -- we compensate for "double counting" the overlap
* The probability of a bike being an even number of years is one minus the probability of being an odd number of years: $P(age \mod 2=0) + P(age \mod 2=1) = 1$
* The probability that my bike is older than ten years old and your bike is *also* older than ten years old $P(mine>10 \land yours>10) = P(mine>10)P(yours>10)$ *assuming* we have absolutely no relation to each other
* The probability of a bike being an even number of years old is given we know it is less than three years old: $P(age \mod 2 = 0|age<3) = P(age \in {0,2}) / P(age \in {0, 1, 2}) = (P(age=0) + P(age=2)) / [P(age=0) + P(age=1) + P(age=2)]$

## Random variables and distributions

* A **random variable** $X$ is a variable whose value is not known, but whose possible values *are* known, and how likely those values are is also known. Probability theory allows us to manipulate random variables without having to assign them a specific value.
* A **probability distribution** $P(X=x)$ associates a random variables outcomes to probabilities. It encodes the plausibility of a variable's outcomes.
* A **probability mass function** $f_X(x)$ is a function that yields probabilities as a function of outcomes.
* If we have uncountable outcomes (like real numbers), we instead use a **probability density function** $f_X(x)$, which just guarantees the rules above hold for dense subsets of the outcomes, even if they can't hold for individual outcomes.
    * Densities are not probabilities! They are non-negative, but can be greater than 1; the *integral* of a density over some domain *is* a probability. (e.g. $P(1\leq age \leq 2) = \int_1^2 f_X(age) dx$ over the interval [1, 2] of $\mathbb{R}$) 

A random variable could represent:

* the outcome of dice throw (discrete), i.e. over the set of outcomes $\{1,2,3,4,5,6\}$; 
* whether or not it is raining outside (discrete: binary), over the set of outcomes $\{\text{heads}, \text{tails}\}$; 
* the height of person we haven't met yet (continuous), over the set of outcomes $\real$; 
* the position of a user's hand (continuous, multi-dimensional), over the set of outcomes $\real^3$. 


## Distributions
A **probability distribution** defines how likely different states of a random variable are. 

We can see $X$ as the the *experiment* and $x$ as the *outcome*, with a function mapping every possible outcome to a probability. 

$$P(X=x),\  \text{the probability of random variable X taking on value x}\\
P(X),\  \text{shorthand for probability of X=x }\\
$$




## Philosophy

We will use the subjective Bayesian interpretation of probability. This has a simple statement but deep implications.

* Probability is a *degree of belief*.
* We express how strongly we believe something to be true with a probability. 
* We encode all beliefs as probability distributions. It's probabilities all the way down.
* We manipulate all beliefs via the rules above. This naturally includes all of classical logic, where P=0 is False and P=1 is True.
* We might expect that these probabilities would be *consistent* with observed relative frequencies of some random repeated process, but **that's not our definition of probability**. We do not invoke the mystical infinitely repeated identical experiments!
* It's completely fine to make statements like "the probability that it is raining right now", "the probability that 10^10^10-1 is prime" or "the probability that the 2012 Olympics was in London" (think carefully about what the probability might be!)
* Because this form of probability theory is merely a logic of uncertain beliefs, we must always reason from some starting point. Rather than **axioms**, as in classical logic, we instead begin from **priors**, associating beliefs to probabilities at the start of a reasoning process.

## Probability mass functions and probability density functions

### Inference

We seek a logical process to perform inference: the deduction of the hidden from the seen. We seek to do so under uncertainty, where we do not deal in absolutes of truth and falsity.

* Our primary tool is Bayes Rule. 
* The ability to do inference is derived from the ability to say: how likely is some unseen X given we saw Y?
    * "How likely is it that my bike is more than four years old given that it has visible corrosion?"
        * I can see corrosion; I can't see age.
    * We can answer that as:
        * It is the probability that we'd observe Y if X were true; multiplied by how likely we *already* believe X to be true; and normalised so that the probability for each possible X sums up to 1.
        * $P(X|Y) = P(Y|X)P(X) / P(Y) = P(Y|X)P(X) / \sum_X P(Y|X)P(X)$
        * $P(X|Y) \propto P(Y|X)P(X)$ if all we care about is how *relatively* likely each possible $X$ is (not how *absolutely* likely it is)
    * These parts have names:
        * `posterior = likelihood * prior / evidence`
        * **posterior** The probability of beliefs about $X$ after having observed $Y$
        * **likelihood** How likely $Y$ is to be observed under any possible hypothesised $X$
        * **prior** How currently likely $X$ is before observing $Y$
        * **evidence** How likely $Y$ is to be observed regardless of what hypothesis we make about $X$

* The likelihood of X, written $L(X)$ is how likely $X$ is to be observed under a particular model. 
    * Probabilities speak of the "future", of the relative propensity for unobserved outcomes to occur.
    * Likelihoods speak of the "past", of observed states. They are just a function of observations/data, and they tell us how likely observed outcomes are under some assumption.
    * The likelihood is $L(x) = f_X(x)$, just the mass/density evaluated for a specific outcome.

## What is a Bayesian? [III]

A **Bayesian**:

* Represents belief exclusively using probability distributions and conducts all computation via the logic of probability.
* Reasons from hypotheses about the world to the evidence that those hypotheses would generate (via a data generating process).
* Updates belief using Bayes' Rule, combining a prior belief with observed evidence to deduce new beliefs.
* Infers conditional distributions -- posterior distributions -- over unseen parameters of the DGP.

Given a parameterised simulator that approximates the problem we are interested in, and some idea about what values these parameters could take on (expressed as a prior probability distribution) we can then use evidence to make a Bayesian update to concentrate a belief distribution on more likely parameter configurations --- a posterior probability distribution.

## A grid model



In [ ]:
import pprint
import itertools


class PMF:
    def __init__(self, pmf):
        # p: outcome -> value
        self.pmf = pmf
        self.outcomes = list(self.pmf.keys())
        self.probs = list(self.pmf.values())

    def __str__(self):
        return pprint.pformat(self.pmf)

    def __repr__(self):
        return str(self)

## Likelihood


In [ ]:
def likelihood(self, outcome):
    return self.pmf[outcome]


PMF.likelihood = likelihood

In [ ]:
def log_lik(self, outcomes):
    return sum([np.log(self.likelihood(o)) for o in outcomes])


PMF.log_lik = log_lik


## Sampling


In [ ]:
def sample(self, n):
    return np.random.choice(self.outcomes, p=self.probs, size=n)


PMF.sample = sample

## Empirical distribution

In [ ]:
from collections import Counter


def from_empirical(self, observations):
    c = Counter(observations)
    total = sum(c.values())
    return PMF({outcome: count / total for outcome, count in c.items()})

## Expectation

In [ ]:
def expect(self, g=lambda x: x):
    return sum(g(outcome) * p for outcome, p in self.pmf.items())


PMF.expect = expect



## Entropy


In [ ]:
def entropy(self):
    return -sum(p * np.log2(p) for p in self.probs)


PMF.entropy = entropy

## Bayes Rule

In [ ]:
def bayes(self, likelihood):
    unnorm = {k: self[k] * likelihood[k] for k in self.pmf}
    s = sum(unnorm.values())
    return PMF({unnorm[k] / s for k in unnorm})


PMF.bayes = bayes

## Joint


In [ ]:
def joint(self, other):
    return PMF(
        {
            (a, b): p_a * p_b
            for (a, p_a), (b, p_b) in itertools.product(
                self.pmf.items(), other.pmf.items()
            )
        }
    )


PMF.__matmul__ = joint

PMF({1: 0.2, 2: 0.8}) @ PMF({"cat": 0.1, "dog": 0.9})

## Marginal

In [ ]:
def marginal(self, n):
    acc = {}
    for outcome, p in self.pmf.items():
        removed = outcome[:n] + outcome[n + 1 :]
        acc[removed] = acc.get(removed, 0) + p
    return PMF(acc)


PMF.marginal = marginal

p = PMF({1: 0.2, 2: 0.8}) @ PMF({"cat": 0.1, "dog": 0.9})

In [ ]:
## Conditional
def conditional(self, n):
    p_B = self.marginal(n)

    for outcome, p in self.pmf.items():
        p / p_B.pmf[outcome[n]]




## Inference approaches

### The process of eliciting, encoding and validating

#### Elicit and encode

#### Validate


### Concrete algorithms

#### MCMC

#### Variational

#### Exact

## What is a Bayesian [IV]

### Why is this Computational HCI?

* We build **statistical models** of user behaviour, and estimate parameters of that model from quantitative observations of data. 
* This is a **model-led approach** which has a strong mathematical underpinning and many powerful algorithmic tools which can be brought to bear.
* This is **robust** (it appropriately represents uncertainty) and **generative** (it can simulate behaviour compatible with observations).  